In [1]:
import requests
from io import StringIO
import pandas as pd

In [2]:
text = """DD	Windrichting (in graden) gemiddeld over de laatste 10 minuten van het afgelopen uur (360=noord, 90=oost, 180=zuid, 270=west, 0=windstil 990=veranderlijk. Zie http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken
FH	Uurgemiddelde windsnelheid (in 0.1 m/s). Zie http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken
FF	Windsnelheid (in 0.1 m/s) gemiddeld over de laatste 10 minuten van het afgelopen uur
FX	Hoogste windstoot (in 0.1 m/s) over het afgelopen uurvak
T	Temperatuur (in 0.1 graden Celsius) op 1.50 m hoogte tijdens de waarneming
T10N	Minimumtemperatuur (in 0.1 graden Celsius) op 10 cm hoogte in de afgelopen 6 uur
TD	Dauwpuntstemperatuur (in 0.1 graden Celsius) op 1.50 m hoogte tijdens de waarneming
SQ	Duur van de zonneschijn (in 0.1 uren) per uurvak, berekend uit globale straling (-1 for <0.05 uur)
Q	Globale straling (in J/cm2) per uurvak
DR	Duur van de neerslag (in 0.1 uur) per uurvak
RH	Uursom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm)
P	Luchtdruk (in 0.1 hPa) herleid naar zeeniveau, tijdens de waarneming
VV	Horizontaal zicht tijdens de waarneming (0=minder dan 100m, 1=100-200m, 2=200-300m,..., 49=4900-5000m, 50=5-6km, 56=6-7km, 57=7-8km, ..., 79=29-30km, 80=30-35km, 81=35-40km,..., 89=meer dan 70km)
N	Bewolking (bedekkingsgraad van de bovenlucht in achtsten), tijdens de waarneming (9=bovenlucht onzichtbaar)
U	Relatieve vochtigheid (in procenten) op 1.50 m hoogte tijdens de waarneming
WW	Weercode (00-99), visueel(WW) of automatisch(WaWa) waargenomen, voor het actuele weer of het weer in het afgelopen uur. Zie http://bibliotheek.knmi.nl/scholierenpdf/weercodes_Nederland
IX	Weercode indicator voor de wijze van waarnemen op een bemand of automatisch station (1=bemand gebruikmakend van code uit visuele waarnemingen, 2,3=bemand en weggelaten (geen belangrijk weersverschijnsel, geen gegevens), 4=automatisch en opgenomen (gebruikmakend van code uit visuele waarnemingen), 5,6=automatisch en weggelaten (geen belangrijk weersverschijnsel, geen gegevens), 7=automatisch gebruikmakend van code uit automatische waarnemingen)
M	Mist 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming
R	Regen 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming
S	Sneeuw 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming
O	Onweer 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming
Y	IJsvorming 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming""" 

data_dict = dict()
for line in text.split("\n"):
    code, description, = line.split("\t")
    data_dict[code] = description

# TODO Split description into name and description

In [3]:
parameters = dict()
parameters["stns"] = "240"
parameters["vars"] = "T:R:O:S:VV:RH"

headers = ["Station", "Date", "Hour"] + parameters["vars"].split(":")
dtypes = {"Station": str, "Date": str, "Hour": int}

In [4]:
url = r"http://projects.knmi.nl/klimatologie/uurgegevens/getdata_uur.cgi"

In [5]:
response = requests.get(url, params=parameters)

In [6]:
response.url

'http://projects.knmi.nl/klimatologie/uurgegevens/getdata_uur.cgi?stns=240&vars=T%3AR%3AO%3AS%3AVV%3ARH'

In [7]:
for line in response.text.split("\n"):
    print(line)

# BRON: KONINKLIJK NEDERLANDS METEOROLOGISCH INSTITUUT (KNMI)
# Opmerking: door stationsverplaatsingen en veranderingen in waarneemmethodieken zijn deze tijdreeksen van uurwaarden mogelijk inhomogeen! Dat betekent dat deze reeks van gemeten waarden niet geschikt is voor trendanalyse. Voor studies naar klimaatverandering verwijzen we naar de gehomogeniseerde reeks maandtemperaturen van De Bilt <http://www.knmi.nl/klimatologie/onderzoeksgegevens/homogeen_260/index.html> of de Centraal Nederland Temperatuur <http://www.knmi.nl/klimatologie/onderzoeksgegevens/CNT/>.
# 
# 
# STN      LON(east)   LAT(north)     ALT(m)  NAME
# 240:         4.790       52.318      -3.30  SCHIPHOL
# 
# YYYYMMDD = datum (YYYY=jaar,MM=maand,DD=dag); 
# HH       = tijd (HH=uur, UT.12 UT=13 MET, 14 MEZT. Uurvak 05 loopt van 04.00 UT tot 5.00 UT; 
# T        = Temperatuur (in 0.1 graden Celsius) op 1.50 m hoogte tijdens de waarneming; 
# R        = Regen 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en

In [8]:
output = StringIO(response.text)

In [9]:
df = pd.read_csv(output, comment="#", names=headers, dtype=dtypes)
df.head()

,Station,Date,Hour,T,R,O,S,VV,RH
0,240,20200301,1,57,1,0,0,75,-1
1,240,20200301,2,51,1,0,0,75,-1
2,240,20200301,3,53,1,0,0,75,-1
3,240,20200301,4,57,0,0,0,75,0
4,240,20200301,5,54,0,0,0,75,0


In [10]:
df["Date"] = pd.to_datetime(df["Date"], format="%Y%m%d")

In [11]:
df.head()

,Station,Date,Hour,T,R,O,S,VV,RH
0,240,2020-03-01,1,57,1,0,0,75,-1
1,240,2020-03-01,2,51,1,0,0,75,-1
2,240,2020-03-01,3,53,1,0,0,75,-1
3,240,2020-03-01,4,57,0,0,0,75,0
4,240,2020-03-01,5,54,0,0,0,75,0
